# HBNO Shopify Plus Growth Service
## Phase 1: Data Foundation & Current State Analysis

**Objective**: Load, clean, and analyze all available Shopify data to establish current baseline metrics and identify key performance indicators.

**Data Sources**:
- Shopify reports (14 CSV files with 12-month historical data)
- Analysis outputs (JSON files with competitive and opportunity data)
- Existing reports (markdown insights)

**Key Outputs**: Unified dataset with baseline metrics for proposal

In [1]:
import pandas as pd
import numpy as np
import json
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Libraries loaded successfully")
print(f"📅 Analysis Date: {datetime.now().strftime('%B %d, %Y at %I:%M %p')}")

✅ Libraries loaded successfully
📅 Analysis Date: November 14, 2025 at 12:59 AM


### 1. Load and Explore Shopify Data

Loading all 14 Shopify report CSV files from the shopify-reports folder.

In [2]:
# Define paths
shopify_reports_path = "/Users/ali/Sites/business/oil-company/project-techteam/shopify-reports/"
analysis_outputs_path = "/Users/ali/Sites/business/oil-company/hbno_analysis_outputs/"
reports_path = "/Users/ali/Sites/business/oil-company/reports/"

# Load all Shopify CSV files
shopify_files = glob.glob(os.path.join(shopify_reports_path, "*.csv"))
shopify_data = {}

print(f"📁 Loading Shopify Reports from: {shopify_reports_path}")
print(f"Found {len(shopify_files)} CSV files\n")

for file in sorted(shopify_files):
    filename = os.path.basename(file)
    df = pd.read_csv(file)
    # Use filename without extension as key
    key = filename.replace(".csv", "")
    shopify_data[key] = df
    print(f"✅ {key}")
    print(f"   Shape: {df.shape} | Columns: {list(df.columns[:3])}...")
    print()

📁 Loading Shopify Reports from: /Users/ali/Sites/business/oil-company/project-techteam/shopify-reports/
Found 14 CSV files

✅ Average order value over time - 2024-11-13 - 2025-11-13
   Shape: (13, 14) | Columns: ['Month', 'Gross sales', 'Discounts']...

✅ Bounce rate over time - 2024-11-13 - 2025-11-13
   Shape: (13, 5) | Columns: ['Month', 'Bounce rate', 'Month (previous_period)']...

✅ Checkout conversion rate over time - 2024-11-13 - 2025-11-13
   Shape: (13, 11) | Columns: ['Month', 'Sessions that reached checkout', 'Sessions that completed checkout']...

✅ Conversion rate breakdown - 2025-08-15 - 2025-11-13
   Shape: (91, 17) | Columns: ['Day', 'Sessions', 'Sessions with cart additions']...

✅ Customer behavior - 2024-11-13 - 2025-11-13
   Shape: (1, 3) | Columns: ['Sessions with cart additions', 'Sessions that reached checkout', 'Sessions that completed checkout']...

✅ New customer sales over time - 2024-11-13 - 2025-11-13
   Shape: (13, 12) | Columns: ['New or returning custome

In [3]:
# Load analysis outputs
print(f"📊 Loading Analysis Outputs from: {analysis_outputs_path}\n")

analysis_outputs = {}
json_files = glob.glob(os.path.join(analysis_outputs_path, "*.json"))

for file in sorted(json_files):
    filename = os.path.basename(file)
    try:
        with open(file, 'r') as f:
            data = json.load(f)
            key = filename.replace(".json", "")
            analysis_outputs[key] = data
            print(f"✅ {filename}")
    except Exception as e:
        print(f"⚠️  Error loading {filename}: {str(e)}")

# Also check for CSV outputs
csv_outputs = glob.glob(os.path.join(analysis_outputs_path, "*.csv"))
print(f"\n📄 Found {len(csv_outputs)} CSV files in analysis outputs")
for file in sorted(csv_outputs):
    filename = os.path.basename(file)
    print(f"   - {filename}")

📊 Loading Analysis Outputs from: /Users/ali/Sites/business/oil-company/hbno_analysis_outputs/

✅ hbno_analysis_report.json
✅ hbno_comprehensive_analysis_report.json
✅ hbno_enhanced_comprehensive_analysis.json
✅ hbno_ultimate_comprehensive_analysis_fast.json
✅ seo_opportunities_analysis.json
✅ shopify_data_summary.json
✅ ultimate_shopify_data_summary.json

📄 Found 4 CSV files in analysis outputs
   - hbno_enhanced_service_proposals.csv
   - hbno_service_proposals.csv
   - hbno_ultimate_service_proposals.csv
   - hbno_ultimate_service_proposals_optimized.csv


### 2. Data Cleaning & Preparation

Standardize data formats and prepare datasets for analysis.

In [4]:
# Function to standardize date columns
def parse_date_flexible(date_str):
    """Try multiple date formats"""
    if pd.isna(date_str):
        return pd.NaT
    formats = ['%Y-%m-%d', '%m/%d/%Y', '%d-%m-%Y', '%B %d, %Y']
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except:
            continue
    return pd.to_datetime(date_str, errors='coerce')

# Clean Shopify data
print("🧹 Cleaning Shopify datasets...\n")

cleaned_data = {}
for key, df in shopify_data.items():
    df_clean = df.copy()
    
    # Try to find and parse date columns
    for col in df_clean.columns:
        if 'date' in col.lower() or 'time' in col.lower():
            df_clean[col] = df_clean[col].apply(parse_date_flexible)
    
    # Remove duplicates
    df_clean = df_clean.drop_duplicates()
    
    # Store cleaned version
    cleaned_data[key] = df_clean
    print(f"✅ {key}: {df_clean.shape[0]} rows after cleaning")

print(f"\n📊 Datasets ready: {list(cleaned_data.keys())}")

🧹 Cleaning Shopify datasets...

✅ Average order value over time - 2024-11-13 - 2025-11-13: 13 rows after cleaning
✅ Bounce rate over time - 2024-11-13 - 2025-11-13: 13 rows after cleaning
✅ Checkout conversion rate over time - 2024-11-13 - 2025-11-13: 13 rows after cleaning
✅ Conversion rate breakdown - 2025-08-15 - 2025-11-13: 91 rows after cleaning
✅ Customer behavior - 2024-11-13 - 2025-11-13: 1 rows after cleaning
✅ New customer sales over time - 2024-11-13 - 2025-11-13: 13 rows after cleaning
✅ New customers over time - 2024-11-13 - 2025-11-13: 13 rows after cleaning
✅ New vs returning customer sales - 2024-11-13 - 2025-11-13: 26 rows after cleaning
✅ New vs returning customers - 2024-11-13 - 2025-11-13: 2 rows after cleaning
✅ Returning customer rate over time - 2024-11-13 - 2025-11-13: 13 rows after cleaning
✅ Sales attributed to marketing - 2024-11-13 - 2025-11-13: 41 rows after cleaning
✅ Sessions by location - 2024-11-13 - 2025-11-13: 13705 rows after cleaning
✅ Sessions by r

### 3. Establish Baseline Metrics

Extract key performance indicators from available data.

In [5]:
# Extract baseline metrics
baseline_metrics = {}

# Average Order Value
if 'Average order value over time - 2024-11-13 - 2025-11-13' in cleaned_data:
    aov_df = cleaned_data['Average order value over time - 2024-11-13 - 2025-11-13']
    if not aov_df.empty:
        # Look for numeric columns
        numeric_cols = aov_df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            avg_aov = aov_df[numeric_cols[0]].mean()
            baseline_metrics['avg_order_value'] = float(avg_aov)
            print(f"💰 Average Order Value: ${avg_aov:.2f}")

# Conversion Rate
if 'Checkout conversion rate over time - 2024-11-13 - 2025-11-13' in cleaned_data:
    conv_df = cleaned_data['Checkout conversion rate over time - 2024-11-13 - 2025-11-13']
    if not conv_df.empty:
        numeric_cols = conv_df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            avg_conv = conv_df[numeric_cols[0]].mean()
            baseline_metrics['avg_conversion_rate'] = float(avg_conv)
            print(f"📊 Average Conversion Rate: {avg_conv:.2%}")

# Bounce Rate
if 'Bounce rate over time - 2024-11-13 - 2025-11-13' in cleaned_data:
    bounce_df = cleaned_data['Bounce rate over time - 2024-11-13 - 2025-11-13']
    if not bounce_df.empty:
        numeric_cols = bounce_df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            avg_bounce = bounce_df[numeric_cols[0]].mean()
            baseline_metrics['avg_bounce_rate'] = float(avg_bounce)
            print(f"🔄 Average Bounce Rate: {avg_bounce:.2%}")

# New vs Returning Customers
if 'New vs returning customers - 2024-11-13 - 2025-11-13' in cleaned_data:
    customer_df = cleaned_data['New vs returning customers - 2024-11-13 - 2025-11-13']
    if not customer_df.empty:
        print(f"\n👥 Customer Segmentation:")
        print(customer_df.head())

print(f"\n✅ Baseline metrics established: {len(baseline_metrics)} KPIs")

💰 Average Order Value: $92587.75
📊 Average Conversion Rate: 40261.54%
🔄 Average Bounce Rate: 77.08%

👥 Customer Segmentation:
  New or returning customer  Customers  Customers (previous_period)  \
0                       New        763                          781   
1                 Returning        736                          745   

   Customers (previous_period)   
0                     -1.801802  
1                     -0.808625  

✅ Baseline metrics established: 3 KPIs


In [6]:
# Extract detailed metrics from Total Sales by Product
if 'Total sales by product - 2024-11-13 - 2025-11-13' in cleaned_data:
    sales_df = cleaned_data['Total sales by product - 2024-11-13 - 2025-11-13']
    print("🏆 Top Performing Products:")
    print(sales_df.head(10))
    
    # Try to extract total revenue
    numeric_cols = sales_df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        total_revenue = sales_df[numeric_cols[0]].sum()
        baseline_metrics['total_revenue'] = float(total_revenue)
        baseline_metrics['avg_monthly_revenue'] = float(total_revenue / 12)
        print(f"\n💹 Total 12-Month Revenue: ${total_revenue:,.2f}")
        print(f"📈 Average Monthly Revenue: ${total_revenue/12:,.2f}")

# Extract sessions data
if 'Sessions by location - 2024-11-13 - 2025-11-13' in cleaned_data:
    location_df = cleaned_data['Sessions by location - 2024-11-13 - 2025-11-13']
    print("\n🌍 Top Geographic Markets:")
    print(location_df.head(10))

if 'Sessions by referrer - 2024-11-13 - 2025-11-13' in cleaned_data:
    referrer_df = cleaned_data['Sessions by referrer - 2024-11-13 - 2025-11-13']
    print("\n🔗 Top Traffic Sources:")
    print(referrer_df.head(10))

🏆 Top Performing Products:
                       Product title Product vendor   Product type  \
0       Lavender 40/42 Essential Oil       HBNO inc          Blend   
1           Peppermint Essential Oil       HBNO inc  essential oil   
2             Rosemary Essential Oil       HBNO inc  essential oil   
3        Cinnamon Bark Essential Oil       HBNO inc  Essential Oil   
4           Lemongrass Essential Oil       HBNO inc  Essential Oil   
5  Eucalyptus Globulus Essential Oil       HBNO inc  Essential Oil   
6                 Neossance Squalane       HBNO inc  essential oil   
7             Tea Tree Essential Oil       HBNO inc  essential oil   
8         Frankincense Oil (Serrata)       HBNO inc  essential oil   
9   Peppermint Essential Oil Organic       HBNO inc  essential oil   

   Net items sold  Gross sales  Discounts  Returns  Net sales    Taxes  \
0             435     71573.29    -438.93    -5.40   71128.96  1620.73   
1             321     68366.09    -244.44  -934.50   6

### 4. Load Analysis Insights

Incorporate existing analysis and opportunity data.

In [7]:
# Load and display analysis insights
print("📋 Analysis Outputs Summary:\n")

if 'hbno_ultimate_comprehensive_analysis_fast' in analysis_outputs:
    analysis = analysis_outputs['hbno_ultimate_comprehensive_analysis_fast']
    print("✅ Loaded comprehensive analysis")
    
    # Extract key insights if available
    if 'executive_summary' in analysis:
        print("\n🎯 Executive Summary:")
        print(analysis['executive_summary'][:500])

if 'ultimate_shopify_data_summary' in analysis_outputs:
    shopify_summary = analysis_outputs['ultimate_shopify_data_summary']
    print("\n📊 Shopify Data Summary loaded")
    
    # Display structure
    if isinstance(shopify_summary, dict):
        print(f"Keys: {list(shopify_summary.keys())[:10]}")

# Load opportunity analysis
if 'seo_opportunities_analysis' in analysis_outputs:
    opportunities = analysis_outputs['seo_opportunities_analysis']
    print("\n🚀 Opportunity Analysis loaded")
    if isinstance(opportunities, dict):
        print(f"Keys: {list(opportunities.keys())[:10]}")

print("\n✅ Analysis insights integrated")

📋 Analysis Outputs Summary:

✅ Loaded comprehensive analysis

📊 Shopify Data Summary loaded
Keys: ['checkout_conversion_rate_over_time', 'total_sales_by_product', 'sessions_by_referrer', 'conversion_rate_breakdown', 'bounce_rate_over_time', 'customer_behavior', 'sales_attributed_to_marketing', 'sessions_by_location', 'average_order_value_over_time', 'new_vs_returning_customer_sales']

🚀 Opportunity Analysis loaded
Keys: ['detailed_analysis']

✅ Analysis insights integrated


### 5. Create Baseline Summary Report

Compile key baseline metrics for proposal foundation.

In [8]:
# Create comprehensive baseline metrics summary
print("=" * 80)
print("📊 HBNO BASELINE METRICS SUMMARY")
print("=" * 80)
print(f"\nAnalysis Period: November 13, 2024 - November 13, 2025 (12 months)")
print(f"Data Collection Date: {datetime.now().strftime('%B %d, %Y at %I:%M %p')}")

print("\n💰 REVENUE METRICS:")
print("-" * 40)
for key, value in baseline_metrics.items():
    if 'revenue' in key.lower():
        print(f"  {key.replace('_', ' ').title()}: ${value:,.2f}")

print("\n📈 PERFORMANCE METRICS:")
print("-" * 40)
for key, value in baseline_metrics.items():
    if 'rate' in key.lower():
        print(f"  {key.replace('_', ' ').title()}: {value:.2%}")
    elif 'order' in key.lower():
        print(f"  {key.replace('_', ' ').title()}: ${value:.2f}")

print("\n📁 DATA SOURCES LOADED:")
print("-" * 40)
print(f"  Shopify Reports: {len(cleaned_data)} datasets")
print(f"  Analysis Outputs: {len(analysis_outputs)} analysis files")
print(f"  Total Metrics Extracted: {len(baseline_metrics)} KPIs")

print("\n" + "=" * 80)

📊 HBNO BASELINE METRICS SUMMARY

Analysis Period: November 13, 2024 - November 13, 2025 (12 months)
Data Collection Date: November 14, 2025 at 12:59 AM

💰 REVENUE METRICS:
----------------------------------------
  Total Revenue: $10,345.00
  Avg Monthly Revenue: $862.08

📈 PERFORMANCE METRICS:
----------------------------------------
  Avg Order Value: $92587.75
  Avg Conversion Rate: 40261.54%
  Avg Bounce Rate: 77.08%

📁 DATA SOURCES LOADED:
----------------------------------------
  Shopify Reports: 14 datasets
  Analysis Outputs: 7 analysis files
  Total Metrics Extracted: 5 KPIs



In [9]:
# Save baseline metrics for downstream notebooks
output_path = "/Users/ali/Sites/business/oil-company/project-techteam/shopify-plus-growth/output/"

# Ensure output directory exists
os.makedirs(output_path, exist_ok=True)

# Save baseline metrics as JSON
baseline_file = os.path.join(output_path, "baseline_metrics.json")
with open(baseline_file, 'w') as f:
    json.dump(baseline_metrics, f, indent=2, default=str)

print(f"✅ Baseline metrics saved to: {baseline_file}")

# Save cleaned data references
data_summary = {
    'datasets_loaded': len(cleaned_data),
    'dataset_names': list(cleaned_data.keys()),
    'analysis_outputs_loaded': len(analysis_outputs),
    'total_kpis_extracted': len(baseline_metrics)
}

data_summary_file = os.path.join(output_path, "data_summary.json")
with open(data_summary_file, 'w') as f:
    json.dump(data_summary, f, indent=2, default=str)

print(f"✅ Data summary saved to: {data_summary_file}")

# Create a summary for display
summary_text = f"""
## PHASE 1 COMPLETION SUMMARY

**Data Successfully Loaded:**
- ✅ Shopify Reports: {len(cleaned_data)} datasets
- ✅ Analysis Outputs: {len(analysis_outputs)} files  
- ✅ Baseline Metrics: {len(baseline_metrics)} KPIs

**Ready for Next Phase:**
All baseline metrics and data have been extracted and saved.
Proceeding to Phase 2: Opportunity Analysis.

**Next Notebook:** 02_opportunity_analysis.ipynb
"""

print(summary_text)

✅ Baseline metrics saved to: /Users/ali/Sites/business/oil-company/project-techteam/shopify-plus-growth/output/baseline_metrics.json
✅ Data summary saved to: /Users/ali/Sites/business/oil-company/project-techteam/shopify-plus-growth/output/data_summary.json

## PHASE 1 COMPLETION SUMMARY

**Data Successfully Loaded:**
- ✅ Shopify Reports: 14 datasets
- ✅ Analysis Outputs: 7 files  
- ✅ Baseline Metrics: 5 KPIs

**Ready for Next Phase:**
All baseline metrics and data have been extracted and saved.
Proceeding to Phase 2: Opportunity Analysis.

**Next Notebook:** 02_opportunity_analysis.ipynb

